In [1]:
!python -V

Python 3.9.16


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
#import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [11]:
#ds = 'yellow_tripdata_2022-01.parquet'
#ds = 'green_tripdata_2021-01.parquet'
ds = 'yellow_tripdata_2022-02.parquet'

In [12]:
df = pd.read_parquet(f'./data/{ds}')

df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
df['duration'].std()

47.26394076778834

In [13]:
N = float(len(df))
df = df[(df.duration >= 1) & (df.duration <= 60)]
M = float(len(df))
print(M/N)

0.979444397269143


In [ ]:
from sklearn.feature_extraction import DictVectorizer
import pandas as pd

# Assuming your dataframe is called 'df' and contains the pickup and dropoff location IDs

# Selecting only the pickup and dropoff location IDs as features
selected_features = ['PULocationID', 'DOLocationID']
df[selected_features] = df[selected_features].astype(str)
df_selected = df[selected_features]

# Convert the dataframe to a list of dictionaries
dict_list = df_selected.to_dict(orient='records')

# Initialize the DictVectorizer
vectorizer = DictVectorizer(sparse=False)

# Fit the DictVectorizer on the dictionary list
vectorizer.fit(dict_list)

# Get the feature names (column names) from the vectorizer
feature_names = vectorizer.get_feature_names_out()

# Get the number of columns (dimensionality) of the feature matrix
num_columns = len(feature_names)

print("Number of columns in the feature matrix:", num_columns)

In [14]:
from sklearn.feature_extraction import DictVectorizer
import pandas as pd

# Assuming your dataframe is called 'df' and contains the pickup and dropoff location IDs

# Selecting only the pickup and dropoff location IDs as features
selected_features = ['PULocationID', 'DOLocationID']
df[selected_features] = df[selected_features].astype(str)
df_selected = df[selected_features]

# Convert the dataframe to a list of dictionaries
dict_list = df_selected.to_dict(orient='records')

# Initialize the DictVectorizer
#vectorizer = DictVectorizer(sparse=True)

Q5

In [10]:
X_train = vectorizer.fit_transform(dict_list)
target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

6.98619123059128

Q6

In [15]:
X_val = vectorizer.transform(dict_list)
target = 'duration'
y_val = df[target].values
y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.786413522518241

In [ ]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [ ]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [ ]:
len(df_train), len(df_val)

In [ ]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [ ]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [ ]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

In [ ]:
import os

if not os.path.exists('models'):
    os.mkdir('models')
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [ ]:
lr = Lasso(0.01)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)